In [ ]:
# Input: every store in a country
# Output: a shortest cycle that travels through every store once and return back to the starting store 

# Algorithms: Greedy heuristic (suboptimal and efficient), Held-Karp algorithm (optimal but inefficient, O(2^n n^2))
# Genetic algorithm : https://www.youtube.com/watch?v=Sk9QQUGMdY8, https://github.com/Josephbakulikira/Traveling-Salesman-Algorithm

# build distance matrix
# find path
# find routes from Mapbox API for the stores involved in the path
# plot

In [ ]:
from util import load_dataset, get_country_dict, get_local_stores

In [ ]:
df = load_dataset()
country_dict = get_country_dict(df)
netherlands_df = get_local_stores(df, country_code="MY")

In [ ]:
netherlands_df

In [ ]:
from geopy import distance as geopy_dist
import os
import warnings
from mapbox import Directions

def build_distance_matrix(local_store_df):
    with open("./mapbox_api_key.txt") as f:
        api_key = f.readline()
    os.environ['MAPBOX_ACCESS_TOKEN'] = api_key
    service = Directions()
    
    n = len(local_store_df)
    distance_mat = [[0 for _ in range(n)] for _ in range(n)]
    path_mat = [[{"path": None, "duration": None, "code": "NoRoute"} for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        print(i)
        for j in range(n):
            
            if i == j: continue
            
            latlong_i = (local_store_df.iloc[i]["latitude"], local_store_df.iloc[i]["longitude"])
            latlong_j = (local_store_df.iloc[j]["latitude"], local_store_df.iloc[j]["longitude"])
            
            response = service.directions([latlong_i[::-1], latlong_j[::-1]], profile="mapbox/driving")
            
            if response.status_code != 200: raise Exception("Failed to retrieve routes from Mapbox API")
            
            response_json = response.json()
            
            if response_json["code"] == "NoRoute":
                warnings.warn("No route found, use geodesic distance")
                distance_mat[i][j] = geopy_dist.geodesic(latlong_i, latlong_j).km
                path_mat[i][j]["path"] = None                      
                path_mat[i][j]["duration"] = None  
            else:
                distance_mat[i][j] = response_json["routes"][0]["distance"] / 1000        # km
                path_mat[i][j]["path"] = response_json["routes"][0]["geometry"]           # polyline string
                path_mat[i][j]["duration"] = response_json["routes"][0]["duration"] / 60  # minutes
                path_mat[i][j]["code"] = "Ok"
    
    return distance_mat, path_mat

In [ ]:
import itertools
import sys

# https://medium.com/analytics-vidhya/are-you-read-for-solving-the-traveling-salesman-problem-80e3c4ea45fc
# https://github.com/CarlEkerot/held-karp
# https://www.youtube.com/watch?v=-JjA4BLQyqE

def held_karp(distance_mat):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        distance_mat: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(distance_mat)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (distance_mat[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit

            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + distance_mat[m][k], m))
                C[(bits, k)] = min(res)

    # We're interested in all bits but the least significant (the start state)
    bits = (2**n - 1) - 1

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + distance_mat[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits

    # Add implicit start state
    path.append(0)

    return opt, list(reversed(path))

In [ ]:
%%timeit -n1 -r1
distance_mat_nl, path_mat_nl = build_distance_matrix(netherlands_df)

In [ ]:
import numpy as np

np.array(distance_mat_nl).round(2)

In [ ]:
path_mat_nl

In [ ]:
cost, path = held_karp(distance_mat_nl)

In [ ]:
import folium
import math
import json
import requests
import polyline
from itertools import cycle

def build_map(center_idx, path, distance_mat, path_mat, local_store_df):
    def construct_address(store_info):
        address = ""
        for i, key in enumerate(('street_address', 'zip_code', 'city', 'country')):
            try:
                isnan = math.isnan(store_info[key])
            except:
                isnan = False # If input to math.isnan is not a float
            if store_info[key] != "0" and not isnan:
                address += f"{store_info[key]}, " if i != 3 else f"{store_info[key]}."
        return address
    
    def construct_openhours(store_info):
        try:
            isnan = math.isnan(store_info["open_hours"])
        except:
            isnan = False # If input to math.isnan is not a float
        
        if not isnan:
            openhours = {}
            for openhour in store_info["open_hours"].split(", "):
                day, time = openhour.split(" : ")
                openhours[day] = time
            return openhours
        return None
    
    def construct_store_tooltip(store_info, storetype):
        address = construct_address(store_info)
        openhours = construct_openhours(store_info)
        storename = store_info["name"]
        url = store_info["url"]
        tooltip = f"""
            <p><i>{storetype}</i></p>
            <h3>{storename}</h3>
            <hr>
            <p><b>Address</b> : {address}</p>
            <table>
                <tr><th>Opening Hours</th></tr>
                {"".join(f"<tr><td>{day}</td><td>{openhours[day]}</td></tr>" for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] if day in openhours) if openhours != None else f"<tr><td>{'Not available'}</td><td></td></tr>"}
            </table>
            <br>
            <p><b>Check us out on :</b><br><a href="{url}">{url}</a></p>
        """
        return tooltip
    
    def fetch_country_geojson(country_name):
        res = requests.get(f"https://nominatim.openstreetmap.org/search?country={country_name}&polygon_geojson=1&format=json")
        country_geojson = json.loads(res.content.decode())[0]["geojson"]
        return country_geojson
        
    
    folium_map = folium.Map(location=(local_store_df.iloc[center_idx]["latitude"], local_store_df.iloc[center_idx]["longitude"]), control_scale=True)
    folium.GeoJson(fetch_country_geojson(country_name=local_store_df.iloc[0]["country"]), name=local_store_df.iloc[0]["country"], style_function=lambda x: {'fillOpacity': 0.1}).add_to(folium_map)
    
    for idx, store_info in local_store_df.iterrows():
        latlong_store = (store_info["latitude"], store_info["longitude"])
        if idx == center_idx:
            center_tooltip = construct_store_tooltip(store_info, storetype="Moonbucks' Distribution Center")
            folium.Marker(location=latlong_store, popup=folium.Popup(center_tooltip, min_width=300, max_width=300), tooltip=folium.Tooltip(center_tooltip), icon=folium.Icon(color="red", icon="building-o", prefix="fa")).add_to(folium_map)
        else:
            store_tooltip = construct_store_tooltip(store_info, storetype="Moonbucks' Branch")
            folium.Marker(location=latlong_store, popup=folium.Popup(store_tooltip, min_width=300, max_width=300), tooltip=folium.Tooltip(store_tooltip), icon=folium.Icon(color="green", icon="home", prefix="glyphicon")).add_to(folium_map)
    
    for i in range(len(path)):
        if i == len(path) - 1: break   # if this is the last store in the path
        origin_idx = path[i] 
        dest_idx = path[i + 1]
        cur_duration = path_mat[origin_idx][dest_idx]["duration"]
        cur_distance = distance_mat[origin_idx][dest_idx]
        cur_path_coords = polyline.decode(path_mat[origin_idx][dest_idx]["path"], geojson=True) 
        cur_path_geojson = {"type": "LineString", "coordinates": cur_path_coords}
        
        path_popup = f"""
            <h5>From : <i><u>{local_store_df.iloc[origin_idx]["name"]}</u></i> <br>To : <i><u>{local_store_df.iloc[dest_idx]["name"]}</u></i></h5>
            <b>Distance : </b> {round(cur_distance, 2)} km <br>
            <b>Duration : </b> {round(cur_duration, 2)} minutes
        """
        folium.GeoJson(cur_path_geojson, style_function=lambda x: {'color': "green"}, name=f"{local_store_df.iloc[origin_idx]['name']} -> {local_store_df.iloc[dest_idx]['name']}").add_child(folium.Popup(path_popup, max_width=300)).add_to(folium_map)
    
    folium.LayerControl().add_to(folium_map)
    return folium_map

In [ ]:
folium_map = build_map(center_idx=0, path=path, distance_mat=distance_mat_nl, path_mat=path_mat_nl, local_store_df=netherlands_df)

In [ ]:
folium_map

In [ ]:
np.array(path_mat_nl)[0][1]

In [ ]:
np.array(distance_mat_nl)[0][1]

In [ ]:
list(folium.map.Icon.color_options)

In [ ]:
from itertools import cycle

color_iter = cycle(folium.map.Icon.color_options)

In [ ]:
abc  = next(color_iter)
print(abc)
print(abc)